## Read data from train.txt and filter it from unwanted patterns


In [66]:
import numpy as np
from keras.utils import to_categorical
from Embeddings import Word2Vec
from Preprocessing import utils, character_encoding
from Models import rnn
# from CharacterEmbeddings import TF_IDF, WordToVec

## Preprocessing


In [67]:
training_set = utils.read_data(f"./Dataset/train.txt")
print("training_set", training_set[0:500])
filtered_training_set = utils.filter_data(training_set)
print("filtered_training_set", filtered_training_set[0:500])


training_set قَوْلُهُ : ( أَوْ قَطَعَ الْأَوَّلُ يَدَهُ إلَخْ ) قَالَ الزَّرْكَشِيُّ( 14 / 123 )
ابْنُ عَرَفَةَ : قَوْلُهُ : بِلَفْظٍ يَقْتَضِيه كَإِنْكَارِ غَيْرِ حَدِيثٍ بِالْإِسْلَامِ وُجُوبَ مَا عُلِمَ وُجُوبُهُ مِنْ الدِّينِ ضَرُورَةً ( كَإِلْقَاءِ مُصْحَفٍ بِقَذَرٍ وَشَدِّ زُنَّارٍ ) ابْنُ عَرَفَةَ : قَوْلُ ابْنِ شَاسٍ : أَوْ بِفِعْلٍ يَتَضَمَّنُهُ هُوَ كَلُبْسِ الزُّنَّارِ وَإِلْقَاءِ الْمُصْحَفِ فِي صَرِيحِ النَّجَاسَةِ وَالسُّجُودِ لِلصَّنَمِ وَنَحْوِ ذَلِكَ ( وَسِحْرٍ ) مُحَمَّدٌ : قَوْلُ مَالِكٍ و
filtered_training_set قَوْلُهُ أَوْ قَطَعَ الْأَوَّلُ يَدَهُ إلَخْ قَالَ الزَّرْكَشِيُّ 
ابْنُ عَرَفَةَ قَوْلُهُ بِلَفْظٍ يَقْتَضِيه كَإِنْكَارِ غَيْرِ حَدِيثٍ بِالْإِسْلَامِ وُجُوبَ مَا عُلِمَ وُجُوبُهُ مِنْ الدِّينِ ضَرُورَةً كَإِلْقَاءِ مُصْحَفٍ بِقَذَرٍ وَشَدِّ زُنَّارٍ ابْنُ عَرَفَةَ قَوْلُ ابْنِ شَاسٍ أَوْ بِفِعْلٍ يَتَضَمَّنُهُ هُوَ كَلُبْسِ الزُّنَّارِ وَإِلْقَاءِ الْمُصْحَفِ فِي صَرِيحِ النَّجَاسَةِ وَالسُّجُودِ لِلصَّنَمِ وَنَحْوِ ذَلِكَ وَسِحْرٍ مُحَمَّدٌ قَوْلُ مَالِكٍ 

In [68]:
words_set = utils.split_data_to_words(filtered_training_set)
text_without_diacritics = []
diacritic_list = []
# Preparing Training Set
text_without_diacritics, diacritic_list = character_encoding.PrepareTrainData(words_set[0:10000])

# Assume this is a test set
text_without_diacritics_test = character_encoding.PrepareTestData(words_set[10000:15000])

In [69]:
# chracter_set = utils.char_tokenizer(filtered_training_set)
# print("chracter_set", chracter_set[0:10])

## Feature Extraction


In [70]:
# Split data to sentences and remove diacritics from each sentence  
sentences = utils.split_data_to_sentences(filtered_training_set)
list_of_sentences = character_encoding.RemoveDiacriticFromSentence(sentences)

In [71]:
# Create Word2Vec model
w2v = Word2Vec.W2V(list_of_sentences, vector_size = 50)
w2v.train()

In [72]:
# Print first 2 words and their vectors
embeddings = list(w2v.Word_embeddings().key_to_index.keys())
for i, word in enumerate(embeddings[0:2]):
    vector = w2v.vector(word)
    print(f"{i}: {word}  - Vector: {vector}")

0: في  - Vector: [-0.19084334  0.33991933  0.8920253   2.038869   -3.0456183   1.2804062
  1.0829108   1.3845755  -1.8281728   0.3677872   2.6614974   0.10976142
 -1.3255929   1.7847508  -3.8392553  -1.8737712  -0.29812673  0.8716539
 -1.2923402  -0.18115766  0.19548194  0.8251692   1.6408223   1.3376774
  0.8235652  -0.40330458  1.4933072   0.7136417  -3.1075103  -0.11752446
 -0.91862994  3.4307263  -0.38698995 -2.2465723  -0.38528436  3.9813666
  3.671119   -1.7181312  -0.3021689  -3.2318962  -1.2201558   1.0487772
  1.3038967  -0.9528995   2.709921    1.9757184  -1.2047977   0.89363015
  1.336037    0.5780968 ]
1: من  - Vector: [-2.951654    0.04070311  0.7041647  -0.22683853  0.15740763  1.086609
  0.47842246  2.594803   -2.088505   -1.1702888   0.66025573  1.3864099
 -1.771956    0.47065756 -1.8496472   2.1469395   0.44570535  1.8917934
 -0.7909366   1.0787189  -0.75279987  2.1971307   2.3592613   0.1185462
  0.82584196 -0.5784354   2.4660666   0.1270661   1.0497872   1.7568791
 -

In [74]:
concatinated_vector = []
for i, word in enumerate(text_without_diacritics):
    # if word does not have corresponding embedding don't add it to the training set and remove its corresponding diacritic list
    if word == "":
        continue
    word_vector = w2v.vector(word)
    for j, char in enumerate(word):
        char_vector = character_encoding.CharToOneHOt(char)
        concatinated_vector.append(np.concatenate((word_vector, char_vector), axis=None))

KeyError: "Key 'والشفعةوإن' not present"

## Building The Model


In [ ]:
input_size = len(concatinated_vector[0])
output_size = len(character_encoding.DIACRITICS)
print("input size : ", input_size)
print("output size : ", output_size)

input size :  36
output size :  15


In [ ]:
# Create an instance of the LSTM class
model = rnn.RNN(input_shape=(None,input_size), output_shape = output_size)

In [ ]:
# Convert the training data to the required format
X_train = concatinated_vector # np.array([[[character_encoding.CharToOneHOt(char)]] for word in text_without_diacritics for char in word])

y_train = []
for word_diacritic in diacritic_list:
    for diacritic in word_diacritic:
        #print(utils.map_text_to_diacritic(diacritic))
        index = character_encoding.DIACRITICS.index(diacritic)
        y_train.append(to_categorical(index, num_classes=output_size))
y_train = np.array(y_train)

In [ ]:
# Train the model
model.train(X_train, y_train, epochs = 5, batch_size = 64)

Epoch 1/5
1236/1236 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.5202 - loss: 1.2761
Epoch 2/5
1236/1236 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.5175 - loss: 1.2825
Epoch 3/5
1236/1236 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.5218 - loss: 1.2774
Epoch 4/5
1236/1236 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.5194 - loss: 1.2801
Epoch 5/5
1236/1236 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.5191 - loss: 1.2807


In [ ]:
# Convert the test data to the required format
X_test = np.array([[character_encoding.CharToOneHOt(char)] for word in text_without_diacritics_test for char in word])

# Predict the diacritics of the test data
y_pred = model.predict(X_test)

611/611 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


## Model Evaluation


In [ ]:
# Evaluate the model
lost , accuracy = model.evaluate(X_test, y_pred)

611/611 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 1.0000 - loss: 1.2780
